<a href="https://colab.research.google.com/github/allensheneka/systemRx/blob/main/systemRx_dataset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##systemRx

Project 2 - Dataset 2

Author:  Sheneka Allen

##Goal

This model will use maintenance dataset to estimate the percent likelihood of Failure (target) of a machine.

##Maintenance Data Set Info

Since real predictive maintenance datasets are generally difficult to obtain and in particular difficult to publish, we present and provide a synthetic dataset that reflects real predictive maintenance encountered in industry to the best of our knowledge.


Attribute Information:

The dataset consists of 10,000 data points stored as rows with 14 features in columns--

>UID: unique identifier ranging from 1 to 10000

>product ID: consisting of a letter L, M, or H for low (50% of all products), medium (30%) and high (20%) as product quality variants and a variant-specific serial number

>air temperature [K]: generated using a random walk process later normalized to a standard deviation of 2 K around 300 K

>process temperature [K]: generated using a random walk process normalized to a standard deviation of 1 K, added to the air temperature plus 10 K.

>rotational speed [rpm]: calculated from a power of 2860 W, overlaid with a normally distributed noise

>torque [Nm]: torque values are normally distributed around 40 Nm with a Ïƒ = 10 Nm and no negative values.

>tool wear [min]: The quality variants H/M/L add 5/3/2 minutes of tool wear to the used tool in the process. 

>machine failure (target): Indicates whether the machine has failed in this particular datapoint for any of the following failure modes are true.  (If at least one of the below failure modes is true, the process fails and the 'machine failure' label is set to 1. It is therefore not transparent to the machine learning method, which of the failure modes has caused the process to fail)

>>The machine failure consists of five independent failure modes:

>>tool wear failure (TWF): the tool will be replaced if fail at a randomly selected tool wear time between 200 and 240 mins (120 times in our dataset). At this point in time, the tool is replaced 69 times, and fails 51 times (randomly assigned).

>>heat dissipation failure (HDF): heat dissipation causes a process failure, if the difference between air- and process temperature is below 8.6 K and the tool's rotational speed is below 1380 rpm. This is the case for 115 data points.

>>power failure (PWF): the product of torque and rotational speed (in rad/s) equals the power required for the process. If this power is below 3500 W or above 9000 W, the process fails, which is the case 95 times in our dataset.

>>overstrain failure (OSF): if the product of tool wear and torque exceeds 11,000 minNm for the L product variant (12,000 M, 13,000 H), the process fails due to overstrain. This is true for 98 datapoints.

>>random failures (RNF): each process has a chance of 0,1 % to fail regardless of its process parameters. This is the case for only 5 datapoints, less than could be expected for 10,000 datapoints in our dataset.

Second choice: dataset 2

1) Source of data: https://archive.ics.uci.edu/ml/datasets/AI4I+2020+Predictive+Maintenance+Dataset#

2) Brief description of data: a synthetic dataset that reflects real predictive maintenance encountered in industry to the best of source UCI's knowledge.

3) What is the target? Machine failure

4) Is this a classification or regression problem? classification

5) How many features? 14

6) How many rows of data? 10,000

7) What, if any, challenges do your foresee in cleaning, exploring, or modeling with this dataset?  I anticipate few challenges with this data (i.e., spelling errors) since this data was manufactured

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn to make a nice heatmap for my confusion matrix
import seaborn as sns

#import some necessary tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

#import accuracy, precision, recall, classification report, and confusion matrix scoring functions
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix

#Importing the Classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# upload Predictive Maintenance data file 

filename = ('/content/drive/MyDrive/Data Science/ai4i2020.csv')
pDm_df = pd.read_csv(filename)
pDm_df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [9]:
pDm_df.duplicated().sum()

0

In [7]:
pDm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      10000 non-null  int64  
 1   Product ID               10000 non-null  object 
 2   Type                     10000 non-null  object 
 3   Air temperature [K]      10000 non-null  float64
 4   Process temperature [K]  10000 non-null  float64
 5   Rotational speed [rpm]   10000 non-null  int64  
 6   Torque [Nm]              10000 non-null  float64
 7   Tool wear [min]          10000 non-null  int64  
 8   Machine failure          10000 non-null  int64  
 9   TWF                      10000 non-null  int64  
 10  HDF                      10000 non-null  int64  
 11  PWF                      10000 non-null  int64  
 12  OSF                      10000 non-null  int64  
 13  RNF                      10000 non-null  int64  
dtypes: float64(3), int64(9)